In [1]:
import os
from smolagents import AzureOpenAIServerModel

model = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
    api_version=os.environ.get("OPENAI_API_VERSION", "")
)

In [2]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "icu_course").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [3]:
from medminer.task.procedure import procedure_task

for doc in docs:
    procedure_task.run(model, doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: procedure                                                                                            │
│ Prompt:                                                                                                         │
│     Given a medical course of a patient, extract all given procedures and save all information as csv. The      │
│ procedures can be in any language. The medical course is usually in the format of bullet points. Every          │
│ procedure should have a single row, if there are multiple procedures that can be extracted from a single piece  │
│ of text, split them up. These are the steps you should follow to complete the task:                             │
│                                                                                                                 │
│     1. Extract a part of the text that contains a procedure. The procedure can be in any language. This is the  │
│ `procedure_reference` column.                                                                                   │
│     2. Translate the procedures to English if necessary and infer the `procedure_translated` column.            │
│     3. Extract the relevant procedure as a string and remove everything that is not relevant. This is the       │
│ `procedure` column.                                                                                             │
│     4. Extract the date of the procedure. If not applicable, write an empty string. This is the `date` column.  │
│     5. Use the `search_snomed_procedures` tool to find SNOMED CT concepts for the extracted procedures. Add the │
│ SNOMED CT ID and fully specified name (FSN) to the output. You will get back a list of dictionaries with the    │
│ following keys: ids and fsn. Usually the first candidate is the best choice, but you can decide otherwise if    │
│ you have reasonable grounds for another decision if you compare the extracted information with the returned     │
│ descriptions from the snomed server. If there are no codes, write an empty string. This is the `snomed_id` and  │
│ `snomed_fsn` columns.                                                                                           │
│                                                                                                                 │
│     Example 1:                                                                                                  │
│     Input: "Colonoscopy performed on 2023-04-15"                                                                │
│     Output: [                                                                                                   │
│         {"patient_id": 1, "procedure_reference": "Colonoscopy performed on 2023-04-15", "procedure_translated": │
│ "Colonoscopy", "procedure": "Colonoscopy", "date": "2023-04-15", "snomed_id": "73761001", "snomed_fsn":         │
│ "Colonoscopy (procedure)"}                                                                                      │
│     \]                                                                                                          │
│                                                                                                                 │
│     Example 2:                                                                                                  │
│     Input: "Gastroscopy and biopsy conducted on 2023-03-10"                                                     │
│     Output: [                                                                                                   │
│         {"patient_id": 2, "procedure_reference": "Gastroscopy and biopsy conducted on 2023-03-10",              │
│ "procedure_translated": "Gastroscopy", "procedure": "G

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'search_snomed_procedures' with arguments: {'term': 'TAVI', 'limit': 5, 'semantic_tag': None}     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'id': '773996000', 'term': 'Transcatheter aortic valve implantation', 'fsn': 'Transcatheter aortic 
valve implantation (procedure)'}, {'id': '53001000087104', 'term': 'History of transcatheter aortic valve 
implantation', 'fsn': 'History of transcatheter aortic valve implantation (situation)'}, {'id': '1184601001', 
'term': 'Valve-in-valve TAVI (transcatheter aortic valve implantation)', 'fsn': 'Revision of transcatheter aortic 
valve implantation (procedure)'}]

[Step 1: Duration 5.38 seconds| Input tokens: 1,963 | Output tokens: 257]

...

[Step 7: Duration 4.23 seconds| Input tokens: 15,926 | Output tokens: 965]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "The extracted procedures have been saved as a CSV file │
│ named 'procedure' in the specified directory."}                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The extracted procedures have been saved as a CSV file named 'procedure' in the specified directory.

[Step 8: Duration 0.72 seconds| Input tokens: 19,041 | Output tokens: 998]